In [ ]:
import boto3
import os, struct
from Crypto.Cipher import AES
from Crypto import Random
import boto3
import hashlib

kms = boto3.client('kms')

In [ ]:
KMS_KEY_ID = "alias/22962256"
LOCAL_PASSWORD = "22962256-KEY"
BLOCK_SIZE = 16
CHUNK_SIZE = 64 * 1024

In [ ]:
def aws_encrypt_file(plainchunk):
    response = kms.encrypt(KeyId=KMS_KEY_ID, Plaintext=plainchunk)
    return response["CiphertextBlob"]


def aws_decrypt_file(cipherchunk):
    response = kms.decrypt(CiphertextBlob=cipherchunk)
    return response["Plaintext"]


def local_encrypt_file(plainchunk, encryptor):
    return encryptor.encrypt(plainchunk)


def local_decrypt_file(cipherchunk, decryptor):
    return decryptor.decrypt(cipherchunk)

In [ ]:
def encrypt_file(in_filename, out_filename, mode="local"):
    with open(in_filename, "rb") as infile:
        with open(out_filename, "wb") as outfile:
            filesize = os.path.getsize(in_filename)
            iv = Random.new().read(AES.block_size)
            outfile.write(struct.pack("<Q", filesize))
            outfile.write(iv)

            if mode == "local":
                local_key = hashlib.sha256(LOCAL_PASSWORD.encode("utf-8")).digest()
                encryptor = AES.new(local_key, AES.MODE_CBC, iv)

            while True:
                chunk = infile.read(CHUNK_SIZE)
                if len(chunk) == 0:
                    break
                elif len(chunk) % 16 != 0:
                    chunk += " ".encode("utf-8") * (16 - len(chunk) % 16)
                cipherchunk = (
                    local_encrypt_file(chunk, encryptor)
                    if mode == "local"
                    else aws_encrypt_file(chunk)
                )
                outfile.write(cipherchunk)


def decrypt_file(in_filename, out_filename, mode="local"):
    with open(in_filename, "rb") as infile:
        origsize = struct.unpack("<Q", infile.read(struct.calcsize("Q")))[0]
        iv = infile.read(16)
        
        if mode == "local":
            local_key = hashlib.sha256(LOCAL_PASSWORD.encode("utf-8")).digest()
            decryptor = AES.new(local_key, AES.MODE_CBC, iv)

        with open(out_filename, "wb") as outfile:
            while True:
                chunk = infile.read(CHUNK_SIZE)
                if len(chunk) == 0:
                    break
                plainchunk = (
                    local_decrypt_file(chunk, decryptor)
                    if mode == "local"
                    else aws_decrypt_file(chunk)
                )
                outfile.write(plainchunk)

            outfile.truncate(origsize)

In [ ]:
response = kms.generate_data_key(KeyId="alias/22962256", KeySpec="AES_256")
response